In [1]:
import numpy as np
import scipy
import time
import math
from scipy.stats import norm

import mystic
from mystic.solvers import DifferentialEvolutionSolver, diffev2
from mystic.strategy import Best1Bin
from mystic.monitors import Monitor,VerboseMonitor

from copy import deepcopy

from tomography import *

from NestedForLoop import get_iterator
from pathlib import Path
from scipy.linalg import sqrtm

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import os
import glob

import pandas as pd

from scipy.optimize import least_squares

import fnmatch
from efficiencies import finding_file, get_channels_eff, set_raw_counts
from optimization import Optimizer, function_fidelity, FidelityResults
from constants import *

from densitymatrix import DensityMatrix, apply_unitary_to_dm

from pathlib import Path
import fnmatch

In [2]:
######################################################################################################
#----- COUNTING THE FILES AND SAVING THEM IN AN ARRAY TO MAKES THE REST OF THE ANALYSIS EASIER -------
######################################################################################################

n_files=0
working_dir=r"C:\Users\LauraMartins\Documents\PhD\Lab\Code\Tomographies\StateTomoData"
os.chdir(working_dir)

filenames = [i for i in glob.glob("StateTomo_2Layers*")]
filenames.sort(key=os.path.getmtime)

index_to_file = {}

for index, filename in enumerate(filenames):
    os.chdir(f"{working_dir}\\{filename}")
    filenames_aux=[i for i in glob.glob("StateTomography*")]
    for index_second, filenames_aux_second in enumerate(filenames_aux):
        index_to_file[n_files] = f"{filename}\\{filenames_aux_second}"
        n_files+=1
os.chdir(working_dir)

In [3]:
os.chdir(working_dir)
qubit_number=2
## Defining the columns of the data file we want to use as data to reconstruct the density matrix (eg.: HH HV VH and VV basis)
column_start=12
column_stop=16

state_after=[]
state_after_file=[]

state_before=[]
state_before_file=[]

state = []
state_file=[]
xp_counts_corrected_with_eff=[]

statetomo = []

#####################################################################
#---------------------- STATE TOMOGRAPHY ----------------------------
#####################################################################
for index in range(len(index_to_file)):
    os.chdir(f"{working_dir}\\{index_to_file[index]}\\StateTomo")
    datafiles=[i for i in glob.glob("*")]
                
    ### Calculating the efficiencies of each detector
    efficiencies=get_channels_eff(datafiles, column_start, column_stop, os.getcwd())
    print(efficiencies)

    ### Opening the data files and writing the data in counts_aux array
    counts_aux=set_raw_counts(datafiles, qubit_number, column_start, column_stop, os.getcwd())
    xp_counts=np.array(np.transpose(counts_aux))

    statetomo.append(LRETomography(int(qubit_number), xp_counts))
    statetomo[-1].run(correct_eff=efficiencies, print_nc=False)
#     statetomo[-1].run(print_nc=False)
    xp_counts_corrected_with_eff.append(statetomo[-1].xp_counts)

    ## The 'e' and 'r' serve to distinguish between tomography before and after, respectively
    ## We want to save them in different arrays because we need them for different things
#     elif index_to_file[index][-8]=='e':
#         state_before.append(statetomo[-1])#.state)
#         state_before_file.append(index_to_file[index])
#         #print('\n Fast maximum likelihood estimation: \n', state_before[-1], '\n')

#     elif index_to_file[index][-8]=='r':
#         state_after.append(statetomo[-1])#.state)
#         state_after_file.append(index_to_file[index])
#         #print('\n Fast maximum likelihood estimation: \n', state_after[-1], '\n')
        
    state.append(statetomo[-1])#.state)
    state_file.append(index_to_file[index])

[[  32795.  611406.  151223.    2473.]
 [1123654.   14128.    3713.   90025.]
 [1116508.   18389.    4761.   94786.]
 [  24293.  631693.  161870.    2027.]]
[1.         0.55527957 0.13997911 0.08240766]
I'm correcting efficiencies
[[354087. 432497. 284609. 114948.]
 [627746. 266230. 162461. 184227.]
 [553942. 259466. 139223. 189159.]
 [352996. 451327. 221851. 112268.]]
[1.         0.74626305 0.42786765 0.31798561]
I'm correcting efficiencies
[[  1365. 271132. 171395.   2046.]
 [277718.   3311.    915. 171766.]
 [322057.   1374.   2176. 150230.]
 [  3570. 235232. 199454.    757.]]
[1.         0.84511419 0.61837906 0.53711531]
I'm correcting efficiencies


In [5]:
(1365+271132+171395+2046)/30

14864.6

In [9]:
######################################
#-- DEFINING THE TARGET BELL STATE ---
######################################

# bell=(np.array([1,0,0,0])+np.array([0,0,0,1]))/np.sqrt(2)
# bell=(np.array([1,0,0,0])-np.array([0,0,0,1]))/np.sqrt(2)
bell=(np.array([0,1,0,0])+np.array([0,0,1,0]))/np.sqrt(2)
# bell=(np.array([0,1,0,0])-np.array([0,0,1,0]))/np.sqrt(2)
bellmatrix=np.array(np.outer(bell, np.conjugate(bell)))

In [10]:
# states=state
# for index in range(len(states)):
#     print(np.real(np.round(states[index].state.fidelity(bell),5)))

In [11]:
##########################################################
#----- OPTIMIZATION OF MAX FIDELITY UP TO UNITARIES ------
##########################################################
states=state
bell=(np.array([0,1,0,0])+np.array([0,0,1,0]))/np.sqrt(2)
fid=np.zeros((n_files))
optimized_matrix=np.zeros((n_files,2**qubit_number,2**qubit_number), dtype='complex')

guess=np.array([0, 0, 0])
bounds=[(-np.pi,np.pi)]*3
results = []

opt=Optimizer(guess, function_fidelity, results=FidelityResults)

for index in range(len(states)):
    result=opt.optimize(states[index].state, bell, bounds=bounds)
    results.append(result)

In [12]:
##########################################################
#------------ ERRORS Input with BELL STATE---------------#
##########################################################
bell=(np.array([0,1,0,0])+np.array([0,0,1,0]))/np.sqrt(2)
bellmatrix=np.array(np.outer(bell, np.conjugate(bell)))
error_runs=2

U=[]
bell_aux=[]
target_ini=[]

states=state
states_file=state_file
players=["Bran", "Arya"]

for index in range(len(states)):
    target=bellmatrix
    U.append(results[index].u)
    target_ini.append(np.transpose(np.conjugate(U[-1]))@bellmatrix@U[-1])    
    
    states[index].calculate_fidelity_error(players, error_runs, opt, target, optimization=True, bounds=bounds)
    
    print('file, fidelity, fidelity_mean, fidelity_std: ',
          states_file[index], np.round(states[index].state.fidelity(target_ini[-1]),5), -np.round(states[index].fidelity_mu,5),
          np.round(states[index].fidelity_std,5), '\n')

Simulating new states considering the uncertainties
Optimizing the fidelity between input and target up to a unitary
file, fidelity, fidelity_mean, fidelity_std:  StateTomo_2Layers\StateTomography (0.98112-0j) 0.98085 0.00014 

Simulating new states considering the uncertainties
Optimizing the fidelity between input and target up to a unitary
file, fidelity, fidelity_mean, fidelity_std:  StateTomo_2Layers\StateTomography_20230330 (0.95639+0j) 0.95646 0.00023 

Simulating new states considering the uncertainties
Optimizing the fidelity between input and target up to a unitary
file, fidelity, fidelity_mean, fidelity_std:  StateTomo_2Layers\StateTomography_202303301812 (0.97713-0j) 0.97702 0.00028 



In [13]:
##########################################################
#--------------- ERRORS OUTPUT TO INTPUT ----------------#
##########################################################
U=[]
bell_aux=[]
target_ini=[]

states=state_before

for index in range(len(states)):
    target=state_after[index]
    U.append(np.kron(results[index].u1,results[index].u2))
    target_ini.append(np.transpose(np.conjugate(U[-1]))@state_after[index].state.state@U[-1])
    
    states[index].calculate_fidelity_error_between_2_experimental_matrices(error_runs, players, target, apply_unitary_to_input=True)
    
    print('file, fidelity, fidelity_mean, fidelity_std: ',
          state_after_file[index], np.round(states[index].state.fidelity(target_ini[-1]),5), -np.round(states[index].fidelity_2_experimental_dms_mu,5),
          np.round(states[index].fidelity_2_experimental_dms_std,5), "\n")

In [9]:
bell=(np.array([0,1,0,0])-np.array([0,0,1,0]))/np.sqrt(2)
bellmatrix=np.array(np.outer(bell, np.conjugate(bell)))

result=opt.optimize(state_before[2].state, bell, bounds=bounds)
U[2]=np.kron(result.u1,result.u2)

t=state_after[2].state.state

t_ini=np.linalg.inv(U[2])@state_after[2].state.state@np.linalg.inv(np.transpose(np.conjugate(U[2])))

states[2].calculate_fidelity_error(players, error_runs, opt, t, bounds=bounds)

print('index, fidelity, fidelity_mean, fidelity_std: ',
        state_before_file[index], np.round(states[2].state.fidelity(t_ini),5), np.round(states[2].fidelity_mu,5),
        np.round(states[2].fidelity_std,5))

index, fidelity, fidelity_mean, fidelity_std:  ChannelSTSteeringHonest202205260905\StateTomography202205261159BeforeChannel (0.99634+0j) -0.99667 0.00013


In [36]:
##########################################################
#------------ ERRORS OUTPUT with BELL STATE--------------#
##########################################################
bell=(np.array([0,1,0,0])+np.array([0,0,1,0]))/np.sqrt(2)
bellmatrix=np.array(np.outer(bell, np.conjugate(bell)))
error_runs=30

U=[]
bell_aux=[]
target_ini=[]

states=state_after
players=["Arya", "Cersei"]

guess=np.array([0, 0, 0])
bounds=[(-np.pi,np.pi)]*3
opt=Optimizer(guess, function_fidelity, results=FidelityResults)

for index in range(len(states)):
    target=bellmatrix
    #U.append(results[index].u)
    #target_ini.append(np.transpose(np.conjugate(U[-1]))@bellmatrix@U[-1])    
    
    states[index].calculate_fidelity_error(players, error_runs, opt, target, bounds=bounds)
    
    print('index, fidelity, fidelity_mean, fidelity_std: ',
          state_after_file[index], np.round(states[index].state.fidelity(target),5), np.round(states[index].fidelity_mu,5),
          np.round(states[index].fidelity_std,5))

index, fidelity, fidelity_mean, fidelity_std:  ChannelSTSteeringHonest202205241718\StateTomography202205241843AfterChannel (0.9907-0j) -0.99055 0.0
index, fidelity, fidelity_mean, fidelity_std:  ChannelSTSteeringHonest202205241947\StateTomography202205242247AfterChannel (0.99083+0j) -0.99098 0.0
index, fidelity, fidelity_mean, fidelity_std:  ChannelSTSteeringHonest202205252200\StateTomography202205251816AfterChannel (0.00334-0j) -0.99294 0.0
index, fidelity, fidelity_mean, fidelity_std:  ChannelSTSteeringHonest202205261311\StateTomography202205261508AfterChannel (0.99254-0j) -0.99237 0.0
index, fidelity, fidelity_mean, fidelity_std:  ChannelSTSteeringHonest202205261554\StateTomography202205261814AfterChannel (0.99217-0j) -0.99218 0.0
index, fidelity, fidelity_mean, fidelity_std:  ChannelSTSteeringHonest202205261859\StateTomography202205262046AfterChannel (0.99141-0j) -0.99175 0.0
index, fidelity, fidelity_mean, fidelity_std:  ChannelSTSteeringHonest202205270908\StateTomography202205261

In [37]:
bell=(np.array([0,1,0,0])-np.array([0,0,1,0]))/np.sqrt(2)
bellmatrix=np.array(np.outer(bell, np.conjugate(bell)))
target=bellmatrix

result=opt.optimize(state_before[2].state, bell, bounds=bounds)

t=state_after[2].state.state

states[2].calculate_fidelity_error(players, error_runs, opt, t, bounds=bounds)

print('index, fidelity, fidelity_mean, fidelity_std: ',
        state_after_file[index], np.round(states[2].state.fidelity(target),5), np.round(states[2].fidelity_mu,5),
        np.round(states[2].fidelity_std,5))

index, fidelity, fidelity_mean, fidelity_std:  ChannelSTSteeringHonest202205260905\StateTomography202205261102AfterChannel (0.99285-0j) -0.99999 0.0


In [ ]:
##########################################################
#------------ WRITING THE DATA IN AN EXCEL ---------------
##########################################################

import xlsxwriter
states=state_before

workbook = xlsxwriter.Workbook('fidelities_to_ini_state_errors.xlsx') ### We should write this in another place
 
worksheet = workbook.add_worksheet()

worksheet.write('A1', 'Number')
worksheet.write('B1', 'Folder')
worksheet.write('C1', 'Fidelity')
worksheet.write('D1', 'Fidelity_mean')
worksheet.write('E1', 'Fidelity_uncertainty')

counter=0
for index in range(len(state_before)):
    
    worksheet.write('A'+str(counter+2), counter)
    worksheet.write('B'+str(counter+2), state_before_file[index])
    worksheet.write('C'+str(counter+2), np.real(np.round(states_final[index].fidelity(target[index]),5)))#np.round(states[index].fidelity_to_pure(bell),5)))
    worksheet.write('D'+str(counter+2), np.real(np.round(states_final[index].mu,5)))
    worksheet.write('E'+str(counter+2), np.real(np.round(states_final[index].std,5)))
    counter+=1
        
workbook.close()

In [ ]:
### IF WE WANT TO CHECK THE FIT TO THE UNCERTARTAINTY
### Then we need to calculate the statistic on these simulated fidelities and calculate the standart deviation
### This will be our uncertainty due to statistical errors
def count_elements(seq) -> dict:
    hist = {}
    for i in seq:
        hist[i] = hist.get(i, 0) + 1
    return hist

counted = count_elements(fidelity_sim)
#print(counted)
bin_numb = len(counted)
errorbar_x=np.array(list(counted))
errorbar_y=np.zeros((bin_numb), dtype=int)

for i in range(bin_numb):
    errorbar_y[i]=counted[errorbar_x[i]]
    
#print(errorbar_x)
#print(errorbar_y)
    
def Gauss(x, A, mu, sigm):
    y = A*np.exp(-((x-mu)/sigm)**2/2)
    return y

mu, std = norm.fit(fidelity_sim)
print(mu, std)
parameters, covariance = curve_fit(Gauss, xdata=errorbar_x[-1], ydata=errorbar_y[:-1], bounds=[(0,0.99007,1e-4),(60,0.9903,0.0002)])
fit_A = parameters[0]
fit_B = parameters[1]
fit_C = parameters[2]
print(fit_A, fit_B, fit_C)

xdata= x = np.linspace(0.989, 0.991, 100)
fit_y = Gauss(xdata, fit_A, fit_B, fit_C)
plt.plot(errorbar_x[:-1], errorbar_y[:-1], 'o', label='data')
plt.plot(xdata, fit_y, '-', label='fit')
plt.legend()